In [2]:
%load_ext autoreload
%autoreload 2

import torch
from src.architectures.feature_extractors.mobilenet import MobileNet
from pytorch_lightning import seed_everything

from torchinfo import summary

seed_everything(42)
img = torch.rand(1, 3, 224, 224)

# summary(net, input_data=img)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Global seed set to 42


In [9]:
v2 = MobileNet(3, 0.9, "v2")
v3_small = MobileNet(3, 0.9, "v3_small")
v3_large = MobileNet(3, 1.0, "v3_large")

In [10]:
v2(img).shape

conv_0 torch.Size([1, 32, 112, 112])
bottleneck_0 torch.Size([1, 16, 112, 112])
bottleneck_1 torch.Size([1, 24, 56, 56])
bottleneck_2 torch.Size([1, 24, 56, 56])
bottleneck_3 torch.Size([1, 32, 28, 28])
bottleneck_4 torch.Size([1, 32, 28, 28])
bottleneck_5 torch.Size([1, 32, 28, 28])
bottleneck_6 torch.Size([1, 56, 14, 14])
bottleneck_7 torch.Size([1, 56, 14, 14])
bottleneck_8 torch.Size([1, 56, 14, 14])
bottleneck_9 torch.Size([1, 56, 14, 14])
bottleneck_10 torch.Size([1, 88, 14, 14])
bottleneck_11 torch.Size([1, 88, 14, 14])
bottleneck_12 torch.Size([1, 88, 14, 14])
bottleneck_13 torch.Size([1, 144, 7, 7])
bottleneck_14 torch.Size([1, 144, 7, 7])
bottleneck_15 torch.Size([1, 144, 7, 7])
last_stage torch.Size([1, 1280, 7, 7])


torch.Size([1, 1280, 7, 7])

In [8]:
0.9 * 160

144.0

In [99]:
v3_small(img).shape


torch.Size([1, 1024, 1, 1])

In [100]:
v3_large(img).shape


torch.Size([1, 1280, 1, 1])

: 